<a href="https://colab.research.google.com/github/huzaifa003/urdu_nlp/blob/main/UrduTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install accelerate -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.3 MB/s eta 0:00:00


In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] ='0'

In [3]:
from transformers import AutoTokenizer, AutoModel,DataCollatorForLanguageModeling, AutoModelForMaskedLM
from torch.optim import AdamW

import pandas as pd

In [4]:
df = pd.read_excel("/content/UrduTranslationsFatehMuhammadAndShaikhulHind.xlsx")
df

,SuraID,AyaNo,Arabic Text,Fateh Muhammad Jalandhri,Mehmood ul Hassan,SurahNameE,SurahNameU
0,1,1,بِسۡمِ اللّٰہِ الرَّحۡمٰنِ الرَّحِیۡمِ,شروع اللہ کا نام لے کر جو بڑا مہربان نہایت رحم...,شروع اللہ کے نام سے جو بڑا مہربان نہایت رحم وا...,Al-Fatiha (The Opening),الفاتحۃ
1,1,2,اَلۡحَمۡدُ لِلّٰہِ رَبِّ الۡعٰلَمِیۡنَ ۙ﴿۱﴾,سب طرح کی تعریف اللہ ہی کیلئے ہے جو تمام مخلو...,سب تعریفیں اللہ کے لئے ہیں [۲] جو پالنے والا س...,Al-Fatiha (The Opening),الفاتحۃ
2,1,3,الرَّحۡمٰنِ الرَّحِیۡمِ ۙ﴿۲﴾,بڑا مہربان نہایت رحم والا۔,بےحد مہربان نہایت رحم والا,Al-Fatiha (The Opening),الفاتحۃ
3,1,4,مٰلِکِ یَوۡمِ الدِّیۡنِ ؕ﴿۳﴾,انصاف کے دن کا حاکم۔,مالک روز جزا کا [۴],Al-Fatiha (The Opening),الفاتحۃ
4,1,5,اِیَّاکَ نَعۡبُدُ وَ اِیَّاکَ نَسۡتَعِیۡنُ ؕ﴿۴﴾,اے پروردگار ہم تیری ہی عبادت کرتے ہیں اور تجھی...,تیری ہی ہم بندگی کرتے ہیں اور تجھی سے مدد چاہت...,Al-Fatiha (The Opening),الفاتحۃ
...,...,...,...,...,...,...,...
6231,114,2,مَلِکِ النَّاسِ ۙ﴿۲﴾,یعنی لوگوں کے بادشاہ کی۔,لوگوں کے بادشاہ کی,An-Nas (The mankind),النَّاس
6232,114,3,اِلٰہِ النَّاسِ ۙ﴿۳﴾,لوگوں کے معبود کی۔,لوگوں کے معبود کی [۱],An-Nas (The mankind),النَّاس
6233,114,4,مِنۡ شَرِّ الۡوَسۡوَاسِ ۬ۙ الۡخَنَّاسِ ۪ۙ﴿۴﴾,اس بڑے وسوسہ ڈالنے والے کی برائی سے جو اللہ کا...,بدی سے اُسکی جو پھسلائے اور چھپ جائے [۲],An-Nas (The mankind),النَّاس
6234,114,5,الَّذِیۡ یُوَسۡوِسُ فِیۡ صُدُوۡرِ النَّاسِ ۙ﴿۵﴾,جو لوگوں کے دلوں میں وسوسے ڈالتا ہے۔,وہ جو خیال ڈالتا ہے لوگوں کے دل میں,An-Nas (The mankind),النَّاس


In [5]:
model_name = "urduhack/roberta-urdu-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)  #generating tokenizer from pretrained model
model  = AutoModelForMaskedLM.from_pretrained(model_name)
model.to("cpu");

Some weights of the model checkpoint at urduhack/roberta-urdu-small were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
def tokenize_function(examples):
    return tokenizer(examples, padding = True, truncation=True, max_length = 512)


In [7]:
tokenized_dataset_Fateh = df['Fateh Muhammad Jalandhri'].map(tokenize_function);
tokenized_dataset_Fateh

0       [input_ids, attention_mask]
1       [input_ids, attention_mask]
2       [input_ids, attention_mask]
3       [input_ids, attention_mask]
4       [input_ids, attention_mask]
                   ...             
6231    [input_ids, attention_mask]
6232    [input_ids, attention_mask]
6233    [input_ids, attention_mask]
6234    [input_ids, attention_mask]
6235    [input_ids, attention_mask]
Name: Fateh Muhammad Jalandhri, Length: 6236, dtype: object

In [8]:
tokenized_dataset_Mehmood = df['Mehmood ul Hassan'].map(tokenize_function);
tokenized_dataset_Mehmood

0       [input_ids, attention_mask]
1       [input_ids, attention_mask]
2       [input_ids, attention_mask]
3       [input_ids, attention_mask]
4       [input_ids, attention_mask]
                   ...             
6231    [input_ids, attention_mask]
6232    [input_ids, attention_mask]
6233    [input_ids, attention_mask]
6234    [input_ids, attention_mask]
6235    [input_ids, attention_mask]
Name: Mehmood ul Hassan, Length: 6236, dtype: object

In [9]:
urdu_dataset = pd.concat([tokenized_dataset_Fateh, tokenized_dataset_Mehmood],ignore_index=True)
urdu_dataset.name = "Urdu Translations"
urdu_dataset

0        [input_ids, attention_mask]
1        [input_ids, attention_mask]
2        [input_ids, attention_mask]
3        [input_ids, attention_mask]
4        [input_ids, attention_mask]
                    ...             
12467    [input_ids, attention_mask]
12468    [input_ids, attention_mask]
12469    [input_ids, attention_mask]
12470    [input_ids, attention_mask]
12471    [input_ids, attention_mask]
Name: Urdu Translations, Length: 12472, dtype: object

In [10]:
from transformers import TrainingArguments, Trainer

In [11]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",num_train_epochs=3)

In [12]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,


)

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= tokenized_dataset_Fateh,
    eval_dataset = tokenized_dataset_Mehmood,
    tokenizer=tokenizer,
    data_collator = data_collator


)

In [14]:
print(len(urdu_dataset))
urdu_dataset[0]

12472


{'input_ids': [0, 8441, 466, 333, 688, 866, 309, 430, 1749, 8436, 2929, 4286, 1163, 316, 1842, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [15]:
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,2.865800,5.675262
2,2.397500,5.610729
3,2.209900,5.526851


TrainOutput(global_step=2340, training_loss=2.479841548561031, metrics={'train_runtime': 571.9381, 'train_samples_per_second': 32.71, 'train_steps_per_second': 4.091, 'total_flos': 723132880856832.0, 'train_loss': 2.479841548561031, 'epoch': 3.0})

In [16]:
import pickle
import torch

In [17]:
model.to('cpu')  # Move the model to the CPU

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): 

In [18]:
# pickle.dump(model, open('/content/drive/MyDrive/Urdu_NLP/fine_tuned_model_urdu.pkl', 'wb'))

In [19]:
import torch

In [20]:
# Specify the file path for saving
model_path = '/content/drive/MyDrive/Urdu_NLP/urdu_model.pth'
torch.save(model, model_path)

In [21]:
# Assuming you're loading the object with torch.load
# loaded_object = pickle.load(open("/content/drive/MyDrive/Urdu_NLP/fine_tuned_model_urdu.pkl", 'rb'))

In [22]:
loaded_model = torch.load(model_path, map_location=torch.device('cpu'))

In [24]:
# Define a masked text
masked_text = "شروع <mask> کے نام سے جو بڑا مہربان نہایت رحم وا"
# Tokenize the masked text
inputs = tokenizer(masked_text, return_tensors="pt")
# Assuming 'inputs' is your input tensor
inputs = inputs.to('cpu')  # Move the tensor to the GPU

# Generate predictions
with torch.no_grad():
    outputs = loaded_model(**inputs)

predicted_ids = torch.argmax(outputs.logits, dim=-1)
predicted_tokens = tokenizer.batch_decode(predicted_ids)

print(predicted_tokens)

[' اوراللہ اللہ کے نام سے جو بڑا مہربان نہایت رحم۔ اور']
